In [87]:
import numpy as np
import matplotlib.pyplot as plt

import codecs
import re
import os.path

def load_movies(path2data): # 1 classe par répertoire
    alltxts = [] # init vide
    labs = []
    cpt = 0
    for cl in os.listdir(path2data): # parcours des fichiers d'un répertoire
        for f in os.listdir(path2data+cl):
            txt = open(path2data+cl+'/'+f).read()
            alltxts.append(txt)
            labs.append(cpt)
        cpt+=1 # chg répertoire = cht classe
        
    return alltxts,labs

path = "./datasets/movies/movies1000/"

alltxts_mov,alllabs_mov = load_movies(path)

print('first review')
print(len(alltxts_mov),len(alllabs_mov))
print(alltxts_mov[0])
print(alllabs_mov[0])

print('second review')

print(alltxts_mov[-1])
print(alllabs_mov[-1])

first review
2000 2000
plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
what's the deal ? 
watch the movie and " sorta " find out . . . 
critique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . 
which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn't snag this one correctly . 
they seem to have taken this pretty neat concept , but executed it terribly . 
so what are the problems with the movie ? 
well , its main problem is that it's simply too jumbled . 
it starts off " normal " but then downshifts into this " fantasy " world in which you , as an audie

In [90]:
''' VERSION PLUS EFFICACE'''
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV


npath = "./datasets/movies/movies1000/"
alltxts_mov, alllabs_mov = load_movies(npath)

# Create and fit the TfidfVectorizer on the training data
tfidf = TfidfVectorizer(use_idf=True, norm='l2', smooth_idf=True, lowercase=False)
X = tfidf.fit_transform(alltxts_mov)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, alllabs_mov, test_size=0.20, random_state=12)

# Define the parameter grid for grid search
param_grid = {
    'Cs': [1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}

# Fit the LogisticRegressionCV model on the training data
nlr = LogisticRegressionCV(cv=5, 
                           scoring='accuracy', 
                           random_state=0, 
                           n_jobs=-1,
                           verbose=3,
                           max_iter=1000)

grid_search = GridSearchCV(nlr, param_grid, cv=5, scoring='accuracy', verbose=3, n_jobs=-1)
# Perform grid search
grid_search.fit(X_train, y_train)
print('Done!')


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.5s remaining:    3.8s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.6s finished


Done!


In [91]:
best_nlr = grid_search.best_estimator_
best_score = grid_search.best_score_

# Generate predictions on the test data using the best estimator
y_pred = best_nlr.predict(X_test)

# Evaluate the best estimator
accuracy = accuracy_score(y_test, y_pred)
print("Best Score:", best_score)
print("Best Estimator:", best_nlr)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred, average=None))

Best Score: 0.844375
Best Estimator: LogisticRegressionCV(cv=5, max_iter=1000, n_jobs=-1, random_state=0,
                     scoring='accuracy', verbose=3)
Accuracy: 0.8375
Confusion Matrix:
[[170  37]
 [ 28 165]]
F1 Score: [0.83950617 0.83544304]


In [93]:
# Load test data
path = "./testSentiment.txt"
with open(path, "r", encoding="utf-8") as file:
    alltxts = [line.strip() for line in file]

# Transform the test data using the same vectorizer used for training
X_test_transformed = tfidf.transform(alltxts)

# Make predictions on the test data
pred = best_nlr.predict(X_test_transformed)
print("Predictions:", pred)

sentiment_mapping = {0: "N", 1: "P"}
pred_sentiments = [sentiment_mapping[prediction] for prediction in pred]

output_file_path = "./new_predictions_reviews.txt"

# Write the sentiment predictions to the output file
with open(output_file_path, "w", encoding="utf-8") as output_file:
    for sentiment in pred_sentiments:
        output_file.write(sentiment + "\n")

Predictions: [1 0 0 ... 0 0 0]
